# Presenting monthly updates results



Older notebook. Contains all of the code in one place. Easier for quick overlook of all the code and easier to canabalize code from here. Run Monthly_run_visualization_final.ipynb for a formal monthly run update. 
This notebook needs to be in the main folder of the viewforecasting to properly connect to the .py files located in the Tools.visualization_assetts subfolder. 

In [ ]:
!pip freeze | grep views-mapper2

# Defining universals

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText


import warnings
warnings.filterwarnings('ignore')

#Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
#import views_runs

from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *

#Data fetching of predictions, note the rewrite required to make the .py files work from a different folder, may require change if the organization of the main branch changed
import sys
import os
home = os.path.expanduser("~")
user = os.getlogin()

#MapperTools
import os
import geopandas as gpd
import sqlalchemy as sa
from ingester3.config import source_db_path
from views_mapper2.mapper2 import *
from views_mapper2.dictionary_writer import *
from views_mapper2.BBoxWriter import *
from views_mapper2.label_writer import *

#ensuring querysets exist and are stored 
from Tools.visualization_assetts.monthly_report_visualization_queryset import *
publish_querysets()

#SAR_logo needed for the non-map graphs
from Tools.visualization_assetts.SAR_logo_import import *
SAR_logo = import_SAR_logo()

In [ ]:
#Common parameters:
#THIS IS A CELL THAT HAS THE ADJUSTABLE VARIABLES

#global end of history
EndOfHistory = 511

#which model do you want to be graphed within this notebook (accepts one model only)
#if you want a different model, rerun this entire notebook again with that new name
#note that this will produce a separate folder, so should not be an issue
model_wanted = 'genetic_ensemble'

#current month run description
dev_id = 'fatalities001'
run_id = dev_id
model_attempt = 't01'

#comparison run and month
#ensure that the dev_id you are using has information for EndOfHistory-1 month
dev_id_comparison = 'fatalities001'
run_id_comparison = dev_id_comparison
model_wanted_comparison = 'genetic_ensemble'
model_attempt_comparison = 't01'

#note that pgm models do not have genetic models yet, hopefully that will not be the case in the future
#in the meantime, i separated out the model search for cm and pgm levels predictions to ensure that something is available for both for this run
model_wanted_pgm = "ensemble_cm_calib"
model_wanted_comparison_pgm = 'ensemble_cm_calib'

#general settings

#size of the map
width_global = 20
height_global = 20

#because of the specificities of how pgm maps work require to use plt.savefig rather than built in .save function within mapper
#choose the dpi size of the pgm maps

pgm_dpi_global = 600

#country map levels are the ones per earlier discussion that should have a background because of how they work and greater usefullness of actual map data
#below is the setting for both the background choice
#the transparency level
#the size of custom views experimental labels
#choice is also available for the background choice for pgm maps
#choices below are what I thought looked good for the two levels
country_background_global = 'OpenStreetMap'
country_mapping_transparency_global = 0.8
views_experimental_font_global = 18
pgm_background_country_level = 'StamenTerrainBackground'

#the following allows for changes to the textbox size
#change here to preferred textbox sizes if you do not like the appearance, especially if the width and height changes were made earlier
#note that ame and africa are close enough in size that their textbox size should be the same

def function_for_textbox(location):
    if str(location) in('globe'):
        textbox_font_size = 10
    elif str(location) in ('ame', 'africa'):
        textbox_font_size = 20
    else:textbox_font_size = 15
    return textbox_font_size

#note anything fatality related should have colormap rainbow, change here if you do not wish this option
#dictionary has been also chosen per previous discussions from standard dictionaries available
#change map color is also set at bwr
#however this is a step to change these defaults if you wish to
fatality_colormap = 'rainbow'
fatality_dictionary_cm =  dictionary_stand_2p_10k
fatality_dictionary_pgm = dictionary_stand_2p
change_colormap_global = 'bwr'


#dictionary for dichototomous prediciton outcome
#same at cm and pgm levels
dich_values=np.array([0,0.2,0.4,0.6,0.8,1.0])
dich_ticklabels=[str(tv) for tv in dich_values]
dichotomous_dictionary_cm = dict(zip(dich_ticklabels, dich_values))

dich_values2=np.array([0,0, 0.01, 0.02, 0.03])
dich_ticklabels2=[str(tv) for tv in dich_values2]
dictionary_dichotomous_pgm = dict(zip(dich_ticklabels2, dich_values2))

#for fetching predicitons, currently there is only one thing that is fetched, which is called step_combined
#this can potentially change in the future
#below choose what column name is given and what you want to call them, this should future proof this code as it will allow for renaming of variables
#please note that if you know that something is log transformed, make sure to change the name accordingly
#small note. dichotomous variable does not need to be double logged but continuous does. this allows the use of the default double log appearance which looks better in maps.

prediction_cont_name_default = {'step_combined':'ln1_step_combined_sb'}
prediction_dich_name_default = {'step_combined':'step_combined_sb'}

do_we_need_to_double_log_cont_variable = 'yes'


#below create double_logged names that you want to use, this feeds into a loop that checks for above 'yes' statement, transforms variable and then renames column
#the loop itself rests in the code portion of the code
#please include the single logged variable as the basis for the double logged one
prediction_cont_name_double_logged = []
variable1 = {'old_name': 'ln1_step_combined_sb', 'new_name':'ln2_step_combined_sb'}
#add additional variables as a continued list, eg. 
#variable2 = {'old_name': 'ln1_step_combined_ns', 'new_name':'ln2_step_combined_ns'}
prediction_cont_name_double_logged.append(variable1)
#here is an example of how to continue the list
#prediction_cont_name_double_logged.append(variable2)


#for predictions portion of the code 
#which variables do you want mapped for predictions
variables_wanted_predictions = {'ln2_step_combined_sb'}

variables_wanted_predictions_dich = {'step_combined_sb'}

#steps wanted for prediction (note that these are the steps to graph)
#additionally for the titles, this is the opportunity to give the output a human name
steps_wanted_prediction_fat = {3, 6, 12, 36}
prediction_title_name = 'Predicted fatalities'
prediction_dich_title_name = 'Predicted dichotomous fatalities'

#choices for line, bar, pie charts
#note that dichotomous only has line choices, as that is the only additional graph type requested for this outcome

#choices for the line/bar/pie charts, including dpi and which variable is wanted
#please use a single ln1 variable version for here. explained in the bar and pie section.

line_bar_pie_charts_dpi = 600

#please use note that the below selection takes more than one variable if they exist, so ns and os, however, we can choose one dictionary at a time, as such please choose one type of transformed fatality variable, e.g., ln1 or actual and match the dictionaries and variable transformation accordingly
variables_wanted_predictions_line_bar_pie = {'ln1_step_combined_sb'}
#this is used inside the functions to create top X lists, note that you can use more than one variable
line_bar_pie_variable_transformation = 'ln1'
variables_wanted_predictions_dich_line = {'step_combined_sb'}

#choices for line for cont and dich choice
#assumption that they should be the same so most are the same for the two
#the first is how many top countries do you want, second line asks cumulative from 1 to what do you want to picture
#dictionary chooses what do you want to use for the y-scale as x-scale is time
#titles are self-explanatory 
#line x and y size is choice of font
#line x and y rotation is choice of labels rotation in case of collision in a graph
#line location is the location of the legend 
#line bbox to anchor is where do you want the legend to be anchored to
#i do not recommend changing the parameters below as they have been optimized for appearance, but if you wish to below provides all of the customization wanted
line_fig_size = (20, 15)
line_top = {5, 10}
cumulative_steps_line_charts = {12, 36}
line_dictionary_cm = dictionary_stand_1p_10k
line_dictionary_cm_dich = dichotomous_dictionary_cm
line_title_cm = "Predicted fatalities per month"
line_title_cm_dich = "Predicted fatalities dichotomous per month"
line_x_size = 15
line_y_size = 15
line_x_rotation = 45
line_y_rotation = 0
line_location = 6
line_bbox_to_anchor = (1, 0.5)
line_title_fontsize = 20
line_x_y_labels_fontsize = 20
line_legend_fontsize = 15
#logo is set to be below the graph on right, textbox with description on the left
SAR_logo_x = 0.1
SAR_logo_y = -0.15
SAR_logo_h_w = 0.2
line_textbox_fontsize = 15
line_textbox_position_x = 0.6
line_textbox_position_y = 0
def region_human_name(region_name):
    if region_name == 'globe':
        output = "worldwide"
    elif region_name == "ame":
        output = "in Africa and Middle East"
    elif region_name == "africa":
        output = "in Africa"
    elif region_name == "me":
        output = "in Middle East"
    else: output = f"in countries {region_name}"
    return output


steps_for_dichotomous_line_graphs = {12, 36}

#bar charts
#first choice is which region do you want, i have coded it to allow choices 'globe', 'ame', 'africa', 'me', or a country list
bar_fig_size = (20, 15)
cumulative_steps_bar_charts = {3, 6, 12}
bar_chart_color = 'blue'
bar_chart_title = 'Cumulative predicted fatalities'
bar_chart_title_font = 20
bar_chart_xlabel_font = 20
bar_chart_xtick_font = 15
bar_chart_ylabel_font = 20
bar_chart_ytick_font = 10
bar_textbox_fontsize = 15
bar_textbox_position_x = 0.6
bar_textbox_position_y = 0

bar_SAR_logo_x = 0.1
bar_SAR_logo_y = -0.15
bar_SAR_logo_h_w = 0.2

#note that the bar charts and pie charts are the only ones that need to have graphing of cumulative fatalities. unfortunately, the procedure will vary on whether the variable we are using is not logged or logged. i attempted to make this future proof, but unfortunately, i could not think of a way to do so. so please use single log transformed variable only. dictionary will not be specified either, as the values are automatically transformed into actuals with assumption that the variable is ln1 transformed. 

#pie charts
#which fatality cutoff do you want to use
#which steps do you want to use
#region works same as above
pie_fig_size = (20, 15)
pie_greater_than = {300, 600}
cumulative_steps_pie_charts = {12, 36}
pie_chart_title = 'Cumulative predicted fatalities'
pie_chart_title_font = 25
pie_chart_start_angle = 90
pie_chart_fontsize = 15
pie_chart_labeldistance = 1.05
pie_chart_legend_title = 'Countries'
pie_chart_legend_loc = 6
pie_chart_bbox_to_anchor = (1,0.5)
pie_chart_legend_fontsize = 15
pie_textbox_fontsize = 15
pie_textbox_position_x = 0.6
pie_textbox_position_y = 0.10

pie_SAR_logo_x = 0.0
pie_SAR_logo_y = -0.05
pie_SAR_logo_h_w = 0.2


#change maps
#how many months back do you want to compare the predictions
months_back_comparison_predictions = {1, 3}
#what are the steps you want to map for change maps
steps_wanted_change_maps = {3, 12}



#change maps dictionary
#note that this is consistent with previous examples where the log transformed variable was used
#please therefore make the assignment below for choices for change variable be the ln1 transformed version of the variables
change_raw_values = np.array([-300,-30,-3,3,30,300])
change_values = np.sign(change_raw_values)*np.log(np.abs(change_raw_values)+1)
change_ticklabels = [str(tv) for tv in change_raw_values]
change_dictionary_cm = dict(zip(change_ticklabels, change_values))

change_dictionary_pgm = change_dictionary_cm


#change dictionary for dichotomous outcomes
dich_values_change=np.array([-1.0, -0.8, -0.6, -0.4, -0.2, 0,0.2,0.4,0.6,0.8,1.0])
dich_ticklabels_change=[str(tv) for tv in dich_values_change]
dichotomous_dictionary_change_cm = dict(zip(dich_ticklabels_change, dich_values_change))

dich_values_change2 = np.array([-0.01, 0, 0.01])
dich_ticklabels_change2 = [str(tv) for tv in dich_values_change2]
dichotomous_dictionary_change_pgm = dict(zip(dich_ticklabels_change2, dich_values_change2))


#some specific change map values for cm_change_maps
#note that first converts the original non specific name into a unique identifier
#second dictionary associated a specific variable with what to compare it with
cm_change_variable_cont_conversion = {'step_combined':'ln1_old_sb'}
cm_change_variable_comparison = {'ln1_step_combined_sb':'ln1_old_sb'}

#same for dichotomous
cm_change_variable_dich_conversion = {'step_combined':'old_sb'}
cm_change_variable_comparison_dich = {'step_combined_sb':'old_sb'}



#choice for the feature maps 
#which variables to do you want to be graphed for features 
#note for change variable, choose a ln1 version of the variables
variables_wanted_features_cm_ged = {'ged_sb_ln2', 'ged_ns_ln2', 'ged_os_ln2'}
variables_wanted_features_cm_other = {'vdem_v2x_libdem','gdp_pc_ln1', "wdi_sp_dyn_imrt_in"}
variables_wanted_features_pgm_ged = {'ged_sb_ln2', 'ged_ns_ln2', 'ged_os_ln2'}
variables_wanted_features_cm_change_ged = {'ged_sb_ln1', 'ged_ns_ln1', 'ged_os_ln1'}
variables_wanted_features_pgm_change_ged = {'ged_sb_ln1', 'ged_ns_ln1', 'ged_os_ln1'}

#which months do you want to have a graph of actual fatalities
months_wanted_features = {EndOfHistory, EndOfHistory -1}
#for the features maps, how many months back do you want a comparison
months_back_actual = {1, 3}
#which months do you want graphed
month_wanted_features_change_maps = {EndOfHistory}
#provide a human name for the features variable title
#provide a human name for the features change maps title
features_ged_title_name = 'Actual fatalities'
features_change_title_name = 'Change in actual fatalities'

#custom_dictionaries
#make sure to choose the correct style for the variables you want
#match the names with the names you wish to use below in the defining colormap and dictionary stage
libdem_values=np.array([0,0.2,0.4,0.6,0.8,1.0])
libdem_ticklabels=[str(tv) for tv in libdem_values]
libdem_dictionary = dict(zip(libdem_ticklabels, libdem_values))

gdp_raw_values = np.array([150,500,1500,5000,15000,50000])
gdp_values = np.array(np.log([150,500,1500,5000,15000,50000]))
gdp_ticklabels = [str(tv) for tv in gdp_raw_values]
gdp_dictionary = dict(zip(gdp_ticklabels, gdp_values))

imr_values = np.array([0,20,40,60,80,100])
imr_ticklabels = [str(tv) for tv in imr_values]
imr_dictionary = dict(zip(imr_ticklabels, imr_values))


#input your preferred variable here 
#note that for anything that is fatalities related, we will continue to use the rainbow
#define custom dictionary for the choice of 

#other choices to be defined below
def function_for_non_fatalities(variable_name):
    if variable_name == 'vdem_v2x_libdem': 
        dictionary_run = libdem_dictionary
        cmap_run = 'viridis_r'
        var_name_run = 'liberal democracy index'
    if variable_name == 'gdp_pc_ln1': 
        dictionary_run = gdp_dictionary
        cmap_run = 'viridis_r'
        var_name_run = 'GDP per capita, logged'
    if variable_name == 'wdi_sp_dyn_imrt_in': 
        dictionary_run = imr_dictionary
        cmap_run = 'viridis'
        var_name_run = 'infant mortality rate per 1,000 live births'
    return dictionary_run, cmap_run, var_name_run

print(f'{user}, you have defined the choices for the run, see the output in next cell for opportunity to adjust country list')

In [ ]:
#this will give you the top X country with predicted cumulatie fatalities at cm level for step1 through 36, please make sure that below uses actual and non-transformed value, though should technically still work for ln_variables. order in the list is based on the country numbers, so do not split the list. 
predstore_future =  'cm_' + model_wanted + '_f' + str(EndOfHistory)

predictions_df = pd.DataFrame.forecasts.read_store(predstore_future, run=dev_id)

#Redefine month_id into steps as a index, change the name of some variables for consistency/workability with code
#note step combined is already a log transformed variable that uses state-based violence
predictions_df = predictions_df.reset_index()
predictions_df['step'] = predictions_df['month_id'] - EndOfHistory
predictions_df = predictions_df.set_index(['step', 'country_id'], drop = True)

#number of countries wanted to get the top for
#please change in case of changes in the future to the naming convention, otherwise keep as is
top_wanted = 1
top_countries= give_me_topX_country_id_cumulative(df=predictions_df, time_index = 'step', number_wanted = top_wanted,
                                           variable = 'step_combined', start = 1, end = 36, variable_transformation = 'ln')

print(f'{user}, the following are the top {top_wanted} countries for fatalities listed for fatality steps 1 through 36 {top_countries}. list in the order from highest to lowest.')

In [ ]:
#THIS IS A CELL THAT HAS THE ADJUSTABLE VARIABLES
#THIS IS THE OPPOURTINITY TO choose your countries
#this is a list of countries/regions you definitely want to have mapped
#you can choose the same country as above
#or choose any countriess you want
#note that since geo_coverage for pgm is for AME region only, there is no need to make global level maps
#when we expand, we can let the two equal, for now
#note that for 

#each country or region gets its own map, countries outside of ame dont have pgm level maps so does not make much sense to include here, but have been coded to handle it
geo_coverage_loop = ['globe', 'ame', 'africa', *top_countries, 79, 80]

#please note that country collections below mean slightly different thing from above
#it does not make sense to have less of a region that at least large sections, but you can include a tuple of country lists you want to be graphed
#here as an example but trully not needed
line_chart_region = ['globe', 'ame', 'africa', (79, 80)]
#bar chart at global scale is a bit too large, so globe not included here though coded for
bar_chart_region = {'ame', 'africa', tuple(top_countries), (79, 80)}
pie_chart_region = {'globe', 'ame', 'africa', tuple(top_countries), (79, 80)}

Starting from here, no further adustments are possible
Code writes to dropbox directly

In [ ]:
#create a directory in dropbox (currently writing to desktop
#note to self, remove delete when done trialing and write to desktop
home = os.path.expanduser("~")
user = os.getlogin()
Mydropbox = home + '/Dropbox (ViEWS)/ViEWS/'
#Monthly_updates = Mydropbox + 'DataReleases/MonthlyUpdates/master_visualization_output_folder/'
Monthly_updates = home + '/Desktop/'
label_version = f'{dev_id}_{model_wanted}_{vid2date_version2(EndOfHistory+1)}_{model_attempt}'
master_folder = Monthly_updates + label_version

#new folderpath
make_folders_complete_set(master_folder)

# Features
- GED, libdem, gdp per capita
- GED in rainbow, libdem and gdp per capita in other colormaps to make a distinction for non-fatality based

## cm features maps

In [ ]:
#note to self: SOME code needs to be moved out

features_qs = (Queryset("Monthly_update_features_cm", "country_month"))
features_cm=features_qs.fetch()

#create new variables, note this part to be moved into queryset when new querysetting is ready
features_cm['gdp_pc']=features_cm['wdi_ny_gdp_mktp_kd']/features_cm['wdi_sp_pop_totl']
features_cm['gdp_pc_ln1']=np.log(features_cm['gdp_pc'])

print(f'{user} cm_current_data fetched successfully')

In [ ]:
#obtaining cm level geometries
engine = sa.create_engine(source_db_path)
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)
print(f'{user} gdf_ci_master fetched successfully')

In [ ]:
#cm level features mapping, ged

#prep data
data= features_cm.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#savefolder
features_cm_folder = master_folder+'/Features/Maps/cm/'

#looping informationa
variable_loop = variables_wanted_features_cm_ged
dictionary_run = fatality_dictionary_cm
cmap_run = fatality_colormap
steps_loop = months_wanted_features

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = give_me_violence_string_label_only(variable)
            bbox_run = bbox_from_cid_region(region_name)                       
                        
            #making the choice of textbox size
            textbox_font_size = function_for_textbox(region_name)
                        
            title_run = f'{features_ged_title_name} in {str(vid2date(step))}, {var_name_run}'
            textbox = f'Name: features,\n{var_run}_at_{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{features_cm_folder}GED_cm{var_run_savefile}_{region_name}_{cmap_run}_month{step}.png'
            
            if region_name in ('globe', 'ame', 'africa'):
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_layer(
                    gdf=data_run,
                    map_dictionary=dictionary_run,
                    cmap='binary',
                    edgecolor="black",
                    linewidth=0.5,
                    transparency = 1.0,
                    column=var_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = 1,
                    masking_location = region_name,
                    column=var_run,
                    edgecolor="black",
                    linewidth=1
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
            else:
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = country_mapping_transparency_global,
                    background = country_background_global,
                    masking_location = str(cid2name(region_name)),
                    column=var_run,
                    edgecolor="black",
                    linewidth=1, 
                    views_experimental_labels = views_experimental_font_global
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
print(f'{user}, cm maps for ged completed')

In [ ]:
#cm features map, all others
#prep data
data= features_cm.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#savefolder
features_cm_folder = master_folder+'/Features/Maps/cm/'

#looping information
variable_loop = variables_wanted_features_cm_other
steps_loop = months_wanted_features

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = function_for_non_fatalities(variable)[2]
            cmap_run = function_for_non_fatalities(variable)[1]
            dictionary_run = function_for_non_fatalities(variable)[0]
            
            textbox_font_size = function_for_textbox(region_name)

            var_run_savefile = variable
            bbox_run = bbox_from_cid(region_name)                       
                        
            title_run = f'{var_name_run} in {str(vid2date(step))}'
            textbox = f'Name: features,\n{var_run}_{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{features_cm_folder}{var_run_savefile}_cm_{region_name}_{cmap_run}_month{step}.png'
            
            if region_name in ('globe', 'ame', 'africa'):
                map_run=Mapper2(
                    width=width_global,
                    height= height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_layer(
                    gdf=data_run,
                    map_dictionary=dictionary_run,
                    cmap='binary',
                    edgecolor="black",
                    linewidth=0.5,
                    transparency = 1.0,
                    column=var_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = 1,
                    masking_location = region_name,
                    column=var_run,
                    edgecolor="black",
                    linewidth=1
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
            else:
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = country_mapping_transparency_global,
                    background = country_background_global,
                    masking_location = str(cid2name(region_name)),
                    column=var_run,
                    edgecolor="black",
                    linewidth=1, 
                    views_experimental_labels = views_experimental_font_global
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
print(f'{user}, cm maps for all other variables completed')

## cm change maps
specification: we are shifting by delta 1 month, so change in value from 1 months ago

In [ ]:
#note change maps will denote the following, increase from the past is red (things got worse, more fatalities)
#decrease is blue (things got better, less fatalities)
#no change is white

for delta in months_back_actual:
    for variable in variables_wanted_features_cm_change_ged:
        column = variable
        actual = np.exp(features_cm[column]) - 1
        shifted = np.exp(features_cm[column].groupby(level = 1).shift(periods = delta)) - 1
        change_run_column = f'ln1_delta_{delta}_{column}'
        features_cm[change_run_column] = np.sign(actual-shifted)*np.log(np.abs(actual-shifted)+1)

        data= features_cm.copy()
        gdf = gdf_ci_master.copy()

        data = data.join(gdf.set_index("country_id"))
        gdf = gpd.GeoDataFrame(data, geometry="geom")

        #savefolder
        features_change_cm_folder = master_folder+'/Features/ChangeMaps/cm/'

        #looping information

        variable_loop = {change_run_column}
        steps_loop = month_wanted_features_change_maps

        #custom dictionary for this run
        dictionary_run = change_dictionary_cm
        cmap_run = change_colormap_global


        for step in steps_loop:
            for variable in variable_loop:
                for region_name in geo_coverage_loop:
                    data_run = gdf.loc[step]
                    var_run = variable
                    var_name_run = find_the_violence_type(variable)
                    var_run_savefile = variable
                    bbox_run = bbox_from_cid(region_name)

                    textbox_font_size = function_for_textbox(region_name)

                    title_run = f'{features_change_title_name} from {delta} month ago on {str(vid2date(step))}, {var_name_run}'
                    textbox = f'Name: features,\n{var_run}_at_{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
                    savefile = f'{features_change_cm_folder}ChangeMaps_GED_cm_{variable}_{region_name}_{change_colormap_global}_month{step}.png'

                    if region_name in ('globe', 'ame', 'africa'):
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_layer(
                            gdf=data_run,
                        map_dictionary=dictionary_run,
                            cmap='binary',
                            edgecolor="black",
                            linewidth=0.5,
                            transparency = 1.0,
                            column=var_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = 1,
                            masking_location = region_name,
                            column=var_run,
                            edgecolor="black",
                            linewidth=1
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
                    else:
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = country_mapping_transparency_global,
                            background = country_background_global,
                            masking_location = str(cid2name(region_name)),
                            column=var_run,
                            edgecolor="black",
                            linewidth=1, 
                            views_experimental_labels = views_experimental_font_global
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
print(f'{user}, cm change maps for ged completed')

## pgm level, features maps

In [ ]:
#obtain data
features_pgm_qs = (Queryset("Monthly_update_features_pgm", "priogrid_month")
)

features_pgm=features_pgm_qs.fetch()


print(f'{user} pgm_current_data fetched successfully')

In [ ]:
#fetching geometries
engine = sa.create_engine(source_db_path)
gdf_pid = gpd.GeoDataFrame.from_postgis(
    "SELECT id as priogrid_gid, in_africa, in_me, geom FROM prod.priogrid", 
    engine, 
    geom_col='geom'
)
gdf_pid = gdf_pid.to_crs(4326)

#now grab the country names
pg_country_queryset = (Queryset("m_associate_country_2_priogrid", "priogrid_month"))

pg_country = pg_country_queryset.fetch().reset_index()
#pg_country_cleaned = pg_country.reset_index().drop(['month_id'], axis = 1).drop_duplicates(subset=['priogrid_gid'])

gdf_pid_master = pd.merge(left = gdf_pid, right = pg_country, left_on = 'priogrid_gid', 
                          right_on = 'priogrid_gid', how = 'right').set_index(['month_id', 'priogrid_gid'])


#grab country geometries for borders creation
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

print(f'{user}, pgm geometries fetching completed')

In [ ]:
#data preparation
data = features_pgm.copy()
gdf = gdf_pid_master.copy()
data_pid = pd.merge(left = data, right = gdf, left_index = True, right_index = True, how = 'left')
gdf = gpd.GeoDataFrame(data_pid, geometry="geom")
gdf_c = gdf_ci_master.copy()

#savefolder
features_pgm_folder = '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/'

#looping informationa
variable_loop = variables_wanted_features_pgm_ged
dictionary_run = fatality_dictionary_pgm
cmap_run = fatality_colormap
steps_loop = months_wanted_features

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = give_me_violence_string_label_only(variable)
            bbox_run = bbox_from_cid_region(region_name)                       
                        
            #making the choice of textbox size
            textbox_font_size = function_for_textbox(region_name)
            
            #masking_choices
            if str(region_name) in('globe'):
                masking_run = 'globe'
                background2 = None
            elif str(region_name) in ('ame'):
                masking_run = 'ame'
                background2 = None
            elif str(region_name) in ('africa'):
                masking_run = 'africa'
                background2 = None
            else:
                masking_run = cid2name(region_name)
                background2 = pgm_background_country_level
                        
            title_run = f'{features_ged_title_name} in {str(vid2date(step))}, {var_name_run}'
            textbox = f'Name: features,\n{var_run}_at_{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{features_pgm_folder}GED_pgm{var_run_savefile}_{region_name}_{cmap_run}_month{step}.png'
                  
            masked=Mapper2(
                width=width_global,
                height=height_global,
                frame_on=True,
                title=title_run,
                bbox=bbox_run
            ).add_mask(
                gdf = data_run,
                map_dictionary = dictionary_run,
                cmap = cmap_run,
                transparency = 1,
                masking_location = masking_run,
                column=var_run,
                background = background2,
                edgecolor="black",
                linewidth=0.5
            ).add_views_textbox(
                text=textbox,
                textsize=textbox_font_size)
            
            ax = masked.ax
            masked_with_borders=gdf_c.plot(ax=ax,edgecolor='black',linewidth=2.0,facecolor='None')
                
            plt.savefig(savefile, dpi = pgm_dpi_global)
            plt.close()
print(f'{user}, pgm maps for ged completed')

## pgm level, features change maps

In [ ]:
#data preparation

for delta in months_back_actual:
    for variable in variables_wanted_features_pgm_change_ged:
        column = variable
        actual = np.exp(features_pgm[column]) - 1
        shifted = np.exp(features_pgm[column].groupby(level = 1).shift(periods = delta)) - 1
        change_run_column = f'ln1_delta_{delta}_{column}'
        features_pgm[change_run_column] = np.sign(actual-shifted)*np.log(np.abs(actual-shifted)+1)
        
        data = features_pgm.copy()
        gdf = gdf_pid_master.copy()
        data_pid = pd.merge(left = data, right = gdf, left_index = True, right_index = True, how = 'left')
        gdf = gpd.GeoDataFrame(data_pid, geometry="geom")
        gdf_c = gdf_ci_master.copy()

        #savefolder
        features_change_pgm_folder = '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/'
        
        #looping information
        variable_loop = {change_run_column}
        steps_loop = month_wanted_features_change_maps

        #custom dictionary for this run
        dictionary_run = change_dictionary_pgm
        cmap_run = change_colormap_global     

        for step in steps_loop:
            for variable in variable_loop:
                for region_name in geo_coverage_loop:
                    data_run = gdf.loc[step]
                    var_run = variable
                    var_name_run = find_the_violence_type(variable)
                    var_run_savefile = give_me_violence_string_label_only(variable)
                    bbox_run = bbox_from_cid_region(region_name)
                    
                    #making the choice of textbox size
                    textbox_font_size = function_for_textbox(region_name)

                    #making the choice of textbox size
                    if str(region_name) in('globe'):
                        masking_run = 'globe'
                        background2 = None
                    elif str(region_name) in ('ame'):
                        masking_run = 'ame'
                        background2 = None
                    elif str(region_name) in ('africa'):
                        masking_run = 'africa'
                        background2 = None
                    else:
                        masking_run = cid2name(region_name)
                        background2 = pgm_background_country_level

                    title_run = f'{features_change_title_name} in {str(vid2date(step))}, {var_name_run}'
                    textbox = f'Name: features,\n{var_run}_at_{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
                    savefile = f'{features_change_pgm_folder}ChangeMaps_GED_pgm{variable}_{region_name}_{cmap_run}_month{step}.png'

                    masked=Mapper2(
                        width=width_global,
                        height=height_global,
                        frame_on=True,
                        title=title_run,
                        bbox=bbox_run
                    ).add_mask(
                        gdf = data_run,
                        map_dictionary = dictionary_run,
                        cmap = cmap_run,
                        transparency = 1,
                        masking_location = masking_run,
                        column=var_run,
                        background = background2,
                        edgecolor="black",
                        linewidth=0.5
                    ).add_views_textbox(
                        text=textbox,
                        textsize=textbox_font_size)

                    ax = masked.ax
                    masked_with_borders=gdf_c.plot(ax=ax,edgecolor='black',linewidth=2.0,facecolor='None')

                    plt.savefig(savefile, dpi = pgm_dpi_global)
                    plt.close()
print(f'{user}, pgm  change maps for ged completed')

# Prediction, cm level

## Continuous and dictotomous mapping, cm level

In [ ]:
#Read in future predictions at cm level as given by the parameters above
#note that the output step combined is already log transformed

predstore_future =  'cm_' + model_wanted + '_f' + str(EndOfHistory)
predstore_future_dich = 'cm_' + model_wanted + '_dich_f' + str(EndOfHistory)

predictions_df = pd.DataFrame.forecasts.read_store(predstore_future, run=dev_id)
predictions_df = predictions_df.reset_index()
predictions_df['step'] = predictions_df['month_id'] - EndOfHistory
predictions_df = predictions_df.set_index(['step', 'country_id'], drop = True)
predictions_df.rename(columns = prediction_cont_name_default, inplace = True)
if do_we_need_to_double_log_cont_variable == 'yes':
    for i in prediction_cont_name_double_logged: predictions_df[i['new_name']]=np.log1p(predictions_df[i['old_name']])


predictions_dich_df = pd.DataFrame.forecasts.read_store(predstore_future_dich, run=dev_id)
predictions_dich_df = predictions_dich_df.reset_index()
predictions_dich_df['step'] = predictions_dich_df['month_id'] - EndOfHistory
predictions_dich_df = predictions_dich_df.set_index(['step', 'country_id'], drop = True)
predictions_dich_df.rename(columns = prediction_dich_name_default, inplace = True)

print(f'{user}, prediction data at cm level successfully fetched')

In [ ]:
#for predictions, we have chosen steps we wanted earlier

#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#savefolder
predictions_cm_folder = master_folder+'/Continuous/Ensemble/ForecastMaps/cm/'

#looping informationa
variable_loop = variables_wanted_predictions
dictionary_run = fatality_dictionary_cm
cmap_run = fatality_colormap
steps_loop = steps_wanted_prediction_fat

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = give_me_violence_string_label_only(variable)
            bbox_run = bbox_from_cid_region(region_name)
            month_id = step+EndOfHistory
                        
            #making the choice of textbox size
            textbox_font_size = function_for_textbox(region_name)
                        
            title_run = f'{prediction_title_name} in {str(vid2date(month_id))}, {var_name_run}'
            textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{predictions_cm_folder}{dev_id}_cm_{var_run_savefile}_{region_name}_{cmap_run}_step{step}.png'
            
            if region_name in ('globe', 'ame', 'africa'):
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_layer(
                    gdf=data_run,
                    map_dictionary=dictionary_run,
                    cmap='binary',
                    edgecolor="black",
                    linewidth=0.5,
                    transparency = 1.0,
                    column=var_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = 1,
                    masking_location = region_name,
                    column=var_run,
                    edgecolor="black",
                    linewidth=1
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
            else:
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = country_mapping_transparency_global,
                    background = country_background_global,
                    masking_location = str(cid2name(region_name)),
                    column=var_run,
                    edgecolor="black",
                    linewidth=1, 
                    views_experimental_labels = views_experimental_font_global
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
print(f'{user}, cm prediction maps completed')

In [ ]:
#maps of the dichotomous predictions
#for predictions, we are interested in steps 3 and 6

#prep data
data= predictions_dich_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#savefolder
predictions_dich_cm_folder = master_folder+'/Dichotomous/Ensemble/ForecastMaps/cm/'

#looping informationa
variable_loop = variables_wanted_predictions_dich
dictionary_run = dichotomous_dictionary_cm
cmap_run = fatality_colormap
steps_loop = steps_wanted_prediction_fat

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = give_me_violence_string_label_only(variable)
            bbox_run = bbox_from_cid_region(region_name)
            month_id = step+EndOfHistory
                        
            #making the choice of textbox size
            textbox_font_size = function_for_textbox(region_name)
                        
            title_run = f'{prediction_dich_title_name} in {str(vid2date(month_id))}, {var_name_run}'
            textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{predictions_dich_cm_folder}{dev_id}_cm_{var_run_savefile}_{region_name}_{cmap_run}_step{step}.png'
            
            if region_name in ('globe', 'ame', 'africa'):
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_layer(
                    gdf=data_run,
                    map_dictionary=dictionary_run,
                    cmap='binary',
                    edgecolor="black",
                    linewidth=0.5,
                    transparency = 1.0,
                    column=var_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = 1,
                    masking_location = region_name,
                    column=var_run,
                    edgecolor="black",
                    linewidth=1
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
            else:
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = country_mapping_transparency_global,
                    background = country_background_global,
                    masking_location = str(cid2name(region_name)),
                    column=var_run,
                    edgecolor="black",
                    linewidth=1, 
                    views_experimental_labels = views_experimental_font_global
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)
                
                map_run.save(savefile)
print(f'{user}, cm dich prediction maps completed')

## Line, Bar and Pie charts, cm level

Continuous gets line, bar and pie charts
Dichomotous gets only line graphs

### line graphs, cm level

In [ ]:
#continuous, linear, cm
ContinuousLineGraph = master_folder + '/Continuous/Ensemble/LineGraphs/'

data = predictions_df.copy()
steps_end_loop = cumulative_steps_line_charts
how_many_countries = line_top
variable_loop = variables_wanted_predictions_line_bar_pie
regions_loop = line_chart_region

for region in regions_loop:
    for top_country in how_many_countries:
        for end_step in steps_end_loop:
            for variable in variable_loop:
                country_name_and_regions_master = Queryset('Monthly_update_country_names_and_regions', 'country_month').fetch().reset_index().drop(columns = ['month_id']).drop_duplicates()

                country_data = country_name_and_regions_master.copy()
                line_charts_data = pd.merge(left = data.reset_index(), right = country_data, left_on = ('country_id'), right_on = ('country_id'), how = 'left').reset_index().set_index(['step', 'country_id'])
                
                if region == 'globe':
                    temp = line_charts_data
                elif region == 'ame':
                    temp = line_charts_data.query('in_africa == 1 or in_me == 1')
                elif region == 'africa':
                    temp = line_charts_data.query('in_africa == 1')
                elif region == 'me':
                    temp = line_charts_data.query('in_me == 1')
                else: temp = line_charts_data.query(f'country_id in {region}')
                
                country_list_run = give_me_topX_country_id_cumulative(df=temp, time_index = 'step', number_wanted = top_country, variable = variable, start = 1, end = end_step, variable_transformation = line_bar_pie_variable_transformation)

                #subset the data
                temporary_df = temp.iloc[temp.index.get_level_values('step') <= end_step]
                data_run = temporary_df[temporary_df.index.get_level_values('country_id').isin(country_list_run)].reset_index().set_index('step')
                #define the tags
                var_name_run = find_the_violence_type(variable)
                var_run_savefile = give_me_violence_string_label_only(variable)

                #define the scale
                dict_run = line_dictionary_cm
                y_label = list(dict_run.keys())
                y_value = list(dict_run.values())

                #define the scale for x acis, and before and after the range of interest

                time_line_values = np.array(range(1, end_step+1, 1))
                time_line_names = [vid2date(i+EndOfHistory) for i in time_line_values]

                savefile = f'{ContinuousLineGraph}LineGraph_continuous_cm{var_run_savefile}_{dev_id}_r{EndOfHistory}_region_{region}_top{top_country}_step1to{end_step}_{variable}.png'

                fig, ax = plt.subplots(figsize = line_fig_size)
                for i in list(data_run.groupby('country_id').groups.keys()):
                    cdata= data_run.groupby('country_id').get_group(i)
                    main = cdata[variable].plot(label=Country(i).name)

                plt.xticks(time_line_values, time_line_names, size = line_x_size, rotation = line_x_rotation)  
                plt.yticks(y_value, y_label, size = line_y_size, rotation = line_y_rotation)
                plt.legend(title='Country', loc=line_location, bbox_to_anchor=line_bbox_to_anchor, fontsize = line_legend_fontsize)

                plt.title(f'{line_title_cm} of top {top_country} countries {region_human_name(region)} for steps 1 through {end_step}, {var_name_run}', fontsize = line_title_fontsize, pad = 20)
                plt.xlabel("Month", fontsize = line_x_y_labels_fontsize)
                plt.ylabel(f"{line_title_cm}, {var_name_run}", fontsize = line_x_y_labels_fontsize)

                logo = fig.add_axes([SAR_logo_x, SAR_logo_y,SAR_logo_h_w, SAR_logo_h_w], anchor='NE', zorder=-1)
                logo.imshow(SAR_logo)
                logo.axis('off')

                props = dict(boxstyle = 'round', facecolor  = 'white', alpha = 0.5)
                textbox = f"https://viewsforecasting.org,\nmodel: {dev_id}_{model_wanted}_{model_attempt},region {region},\n{variable},last input: {str(vid2date(EndOfHistory))}"
                
                plt.text(x = line_textbox_position_x, y = line_textbox_position_y, s = textbox, bbox = props, fontsize = line_textbox_fontsize, transform=plt.gcf().transFigure)
        
                plt.savefig(savefile, dpi = line_bar_pie_charts_dpi, bbox_inches= 'tight')
                plt.close()
       

        
print(f'{user}, line for continuous predictions, cm level completed')
        

In [ ]:
#dichotomous linear graph 

#continuous, linear, cm
DichotomousLineGraph = master_folder + '/Dichotomous/Ensemble/LineGraphs/'

data = predictions_dich_df.copy()
steps_end_loop = cumulative_steps_line_charts
how_many_countries = line_top
variable_loop = variables_wanted_predictions_dich_line
regions_loop = line_chart_region

for region in regions_loop:
    for top_country in how_many_countries:
        for end_step in steps_end_loop:
            for variable in variable_loop:
                country_name_and_regions_master = Queryset('Monthly_update_country_names_and_regions', 'country_month').fetch().reset_index().drop(columns = ['month_id']).drop_duplicates()

                country_data = country_name_and_regions_master.copy()
                line_charts_data = pd.merge(left = data.reset_index(), right = country_data, left_on = ('country_id'), right_on = ('country_id'), how = 'left').reset_index().set_index(['step', 'country_id'])
                
                if region == 'globe':
                    temp = line_charts_data
                elif region == 'ame':
                    temp = line_charts_data.query('in_africa == 1 or in_me == 1')
                elif region == 'africa':
                    temp = line_charts_data.query('in_africa == 1')
                elif region == 'me':
                    temp = line_charts_data.query('in_me == 1')
                else: temp = line_charts_data.query(f'country_id in {region}')
                
                country_list_run = give_me_topX_country_id_cumulative(df=temp, time_index = 'step', number_wanted = top_country, variable = variable, start = 1, end = end_step, variable_transformation = line_bar_pie_variable_transformation)

                #subset the data
                temporary_df = temp.iloc[temp.index.get_level_values('step') <= end_step]
                data_run = temporary_df[temporary_df.index.get_level_values('country_id').isin(country_list_run)].reset_index().set_index('step')
                #define the tags
                var_name_run = find_the_violence_type(variable)
                var_run_savefile = give_me_violence_string_label_only(variable)

                #define the scale
                dict_run = line_dictionary_cm_dich
                y_label = list(dict_run.keys())
                y_value = list(dict_run.values())

                #define the scale for x acis, and before and after the range of interest

                time_line_values = np.array(range(1, end_step+1, 1))
                time_line_names = [vid2date(i+EndOfHistory) for i in time_line_values]

                savefile = f'{DichotomousLineGraph}LineGraph_dichotomous_cm{var_run_savefile}_{dev_id}_r{EndOfHistory}_region_{region}_top{top_country}_step1to{end_step}_{variable}.png'

                fig, ax = plt.subplots(figsize = line_fig_size)
                for i in list(data_run.groupby('country_id').groups.keys()):
                    cdata= data_run.groupby('country_id').get_group(i)
                    cdata[variable].plot(label=Country(i).name)

            plt.xticks(time_line_values, time_line_names, size = line_x_size, rotation = line_x_rotation)  
            plt.yticks(y_value, y_label, size = line_y_size, rotation = line_y_rotation)
            plt.legend(title='Country', loc=line_location, bbox_to_anchor=line_bbox_to_anchor, fontsize = line_legend_fontsize)

            plt.title(f'{line_title_cm} of top {top_country} countries {region_human_name(region)} for steps 1 through {end_step}, {var_name_run}', fontsize = line_title_fontsize, pad = 20)
            plt.xlabel("Month", fontsize = line_x_y_labels_fontsize)
            plt.ylabel(f"{line_title_cm}, {var_name_run}", fontsize = line_x_y_labels_fontsize)
            
            logo = fig.add_axes([SAR_logo_x, SAR_logo_y,SAR_logo_h_w, SAR_logo_h_w], anchor='NE', zorder=-1)
            logo.imshow(SAR_logo)
            logo.axis('off')
            
            props = dict(boxstyle = 'round', facecolor  = 'white', alpha = 0.5)
            textbox = f"https://viewsforecasting.org,\nmodel: {dev_id}_{model_wanted}_{model_attempt},region {region},\n{variable},last input: {str(vid2date(EndOfHistory))}"
            
            plt.text(x = line_textbox_position_x, y = line_textbox_position_y, s = textbox, bbox = props, fontsize = line_textbox_fontsize, transform=plt.gcf().transFigure)

            plt.savefig(savefile, dpi = line_bar_pie_charts_dpi, bbox_inches= 'tight')
            
            plt.close()

        
print(f'{user}, line for dichotomous predictions, cm level completed')
        

### bar charts, cm level

In [ ]:
#continuous, bar graph, cm
ContinuousBarCharts = master_folder + '/Continuous/Ensemble/BarCharts/'

data = predictions_df.copy()
steps_end_loop = cumulative_steps_bar_charts
variable_loop = variables_wanted_predictions_line_bar_pie
regions_loop = bar_chart_region

country_name_and_regions_master = Queryset('Monthly_update_country_names_and_regions', 'country_month').fetch().reset_index().drop(columns = ['month_id']).drop_duplicates()

country_data = country_name_and_regions_master.copy()
bar_charts_data = pd.merge(left = data.reset_index(), right = country_data, left_on = ('country_id'), right_on = ('country_id'), how = 'left').reset_index().set_index(['step', 'country_id'])

for region in regions_loop:
    for end_step in steps_end_loop:
        for variable in variable_loop:
            #choosing a subset of data
            
            if region == 'globe':
                temp = bar_charts_data
            elif region == 'ame':
                temp = bar_charts_data.query('in_africa == 1 or in_me == 1')
            elif region == 'africa':
                temp = bar_charts_data.query('in_africa == 1')
            elif region == 'me':
                temp = bar_charts_data.query('in_me == 1')
            else: temp = bar_charts_data.query(f'country_id in {region}')
            
            temp2 = temp.iloc[temp.index.get_level_values('step')<=end_step]
            temp2['actual'] = np.exp(temp2[variable])-1
            temp3 = pd.DataFrame(temp2.groupby('country_id', dropna = False).agg({str('actual'):'sum'})).reset_index()
            data_run = pd.merge(left = temp3, right = country_data, left_on = 'country_id', right_on = 'country_id', how = 'left')
            
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = give_me_violence_string_label_only(variable)
            
            savefile = f'{ContinuousBarCharts}BarCharts_continuous_cm{var_run_savefile}_{dev_id}_r{EndOfHistory}_region_{region}_step1to{end_step}_{variable}.png' 
            
            name = data_run['name']
            value = data_run['actual']
            
            fig, ax = plt.subplots(figsize = bar_fig_size)
            plt.barh(name, value, color = bar_chart_color)
            plt.title(f'{bar_chart_title} {region_human_name(region)} from step 1 to {end_step}, {var_name_run}', fontsize = bar_chart_title_font, pad = 20)
            plt.xlabel(f'{bar_chart_title}, {var_name_run}', fontsize = bar_chart_xlabel_font)
            plt.xticks(fontsize = bar_chart_xtick_font)
            plt.ylabel('Country', fontsize = bar_chart_ylabel_font)
            plt.yticks(fontsize = bar_chart_ytick_font)
            
            logo = fig.add_axes([bar_SAR_logo_x, bar_SAR_logo_y, bar_SAR_logo_h_w, bar_SAR_logo_h_w], anchor='NE', zorder=-1)
            logo.imshow(SAR_logo)
            logo.axis('off')
            
            props = dict(boxstyle = 'round', facecolor  = 'white', alpha = 0.5)
            textbox = f"https://viewsforecasting.org,\nmodel: {dev_id}_{model_wanted}_{model_attempt},region {region},\n{variable},last input: {str(vid2date(EndOfHistory))}"
            
            plt.text(x = bar_textbox_position_x, y = bar_textbox_position_y, s = textbox, bbox = props, fontsize = bar_textbox_fontsize, transform=plt.gcf().transFigure)
            
            plt.savefig(savefile, dpi = line_bar_pie_charts_dpi, bbox_inches= 'tight')
            plt.close()

print(f'{user}, bar charts completed')

            

### pie charts, cm level

In [ ]:
#pie chart for the continous prediction
ContinuousPieCharts = master_folder + '/Continuous/Ensemble/PieCharts/'

data = predictions_df.copy()
steps_end_loop = cumulative_steps_pie_charts
variable_loop = variables_wanted_predictions_line_bar_pie
regions_loop = pie_chart_region
pie_greater_than_loop = pie_greater_than

country_name_and_regions_master = Queryset('Monthly_update_country_names_and_regions', 'country_month').fetch().reset_index().drop(columns = ['month_id']).drop_duplicates()

country_data = country_name_and_regions_master.copy()
pie_charts_data = pd.merge(left = data.reset_index(), right = country_data, left_on = ('country_id'), right_on = ('country_id'), how = 'left').reset_index().set_index(['step', 'country_id'])

for region in regions_loop:
    for end_step in steps_end_loop:
        for variable in variable_loop:
            for pie_cutoff in pie_greater_than_loop:
            #choosing a subset of data
            
                if region == 'globe':
                    temp = pie_charts_data
                elif region == 'ame':
                    temp = pie_charts_data.query('in_africa == 1 or in_me == 1')
                elif region == 'africa':
                    temp = pie_charts_data.query('in_africa == 1')
                elif region == 'me':
                    temp = pie_charts_data.query('in_me == 1')
                else: temp = pie_charts_data.query(f'country_id in {region}')

                temp2 = temp.iloc[temp.index.get_level_values('step')<=end_step]
                temp2['actual'] = np.exp(temp2[variable])-1
                temp3 = pd.DataFrame(temp2.groupby('country_id', dropna = False).agg({str('actual'):'sum'})).reset_index()
                temp4 = pd.merge(left = temp3, right = country_data, left_on = 'country_id', right_on = 'country_id', how = 'left')
                data_run = temp4.query(f'actual >= {pie_cutoff}')

                var_name_run = find_the_violence_type(variable)
                var_run_savefile = give_me_violence_string_label_only(variable)

                savefile = f'{ContinuousPieCharts}PieCharts_continuous_cm{var_run_savefile}_{dev_id}_r{EndOfHistory}_region_{region}_step1to{end_step}_{variable}_cutoff{pie_cutoff}.png' 
                
                data_run['legend'] = data_run['country_id'].astype(str)+'='+data_run['name']
                
                pie_label = data_run['country_id']
                pie_legend_label = data_run['legend']
                value = data_run['actual']

                fig,ax = plt.subplots(figsize = pie_fig_size)
                pie = plt.pie(value, labels =pie_label, startangle = pie_chart_start_angle, textprops = {'fontsize':pie_chart_fontsize}, labeldistance = pie_chart_labeldistance)
                plt.legend(pie[0], labels = pie_legend_label, title = pie_chart_legend_title, loc = pie_chart_legend_loc, bbox_to_anchor = pie_chart_bbox_to_anchor, fontsize = pie_chart_legend_fontsize)
                plt.title(f'{pie_chart_title} from step 1 to {end_step}, {var_name_run}, cutoff {pie_cutoff} {region_human_name(region)}', fontsize = pie_chart_title_font)
                
                logo = fig.add_axes([pie_SAR_logo_x, pie_SAR_logo_y, pie_SAR_logo_h_w, pie_SAR_logo_h_w], anchor='NE', zorder=-1)
                logo.imshow(SAR_logo)
                logo.axis('off')
                
                props = dict(boxstyle = 'round', facecolor  = 'white', alpha = 0.5)
                textbox = f"https://viewsforecasting.org,\nmodel: {dev_id}_{model_wanted}_{model_attempt},region {region},\ncutoff{pie_cutoff}, {variable},last input: {str(vid2date(EndOfHistory))}"
            
                plt.text(x = pie_textbox_position_x, y = pie_textbox_position_y, s = textbox, bbox = props, fontsize = pie_textbox_fontsize, transform=plt.gcf().transFigure)

                plt.savefig(savefile, dpi = line_bar_pie_charts_dpi, bbox_inches= 'tight')
                plt.close()

print(f'{user}, pie charts completed')

## Continuous and dichotomous mapping at cm level, change

Note for this case, there are several ways to interpret change maps
Version 1: compare step n with step n of previous run
Version 2: compare month n with month n of previous run
Version 3: compare step 1 with the average running of the actuals for step 0 (EndOfHistory)
version4: compare step1 with the last actual for step 0 (EndOfHistory)

### Continuous change maps, version 1 and 2

In [ ]:
#continuous_changes_vs_actuals_or_running_average
data = predictions_df.copy()
historic = Queryset("Monthly_update_features_cm", "country_month").fetch()[['ged_sb_ln1', 'ged_ns_ln1', 'ged_os_ln1']]
end_of_history = historic.query(f'month_id == {EndOfHistory}')
data2 = pd.merge(data.reset_index(), end_of_history, left_on = 'country_id', right_on = 'country_id', how = 'left').set_index(['step', 'country_id'])
rolling_average = Queryset("Monthly_update_running_average_cm", "country_month").fetch()
rolling_average_end_of_history = rolling_average.query(f'month_id == {EndOfHistory}')
data3 = pd.merge(data2.reset_index(), rolling_average_end_of_history, left_on = 'country_id', right_on = 'country_id', how = 'left').set_index(['step', 'country_id'])

for variable, variable_old in cm_change_variable_comparison.items():
    conf = give_me_violence_string_label_only(variable)
    if conf == '_sb': variables_to_compare = {'ged_sb_ln1', 'ged_sb_run_average_6_ln1'}
    elif conf == '_ns': variables_to_compare = {'ged_ns_ln1', 'ged_ns_run_average_6_ln1'}
    elif conf == '_os': variables_compare = {'ged_os_ln1', 'ged_os_run_average_6_ln1'}
    else: print('variable name does not specify violence type, make sure it does')
    column_prediction = variable
    variable_loop = []
    for column_comparison in variables_to_compare:
        actual_prediction = np.exp(data3[column_prediction])-1
        actual_comparison = np.exp(data3[column_comparison])-1
        temp_change_name = f'change_from_{column_comparison}'
        data3[temp_change_name] = np.sign(actual_prediction - actual_comparison)*np.log(np.abs(actual_prediction - actual_comparison)+1)
        variable_loop.append(temp_change_name)

        
#geometries
gdf = gdf_ci_master.copy()
data3 = data3.join(gdf.set_index('country_id'))
gdf = gpd.GeoDataFrame(data3, geometry = "geom")

cm_predictions_change_maps_folder = master_folder + '/Continuous/Ensemble/ChangeMaps/cm/'

steps_loop = steps_wanted_change_maps
dictionary_run = change_dictionary_cm
cmap_run = change_colormap_global

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = variable
            bbox_run = bbox_from_cid(region_name)
            month_id_temp = step+EndOfHistory

            textbox_font_size = function_for_textbox(region_name)

            if 'ged_sb_ln1' or 'ged_ns_ln1' or 'ged_os_ln1' in variable: human_name = 'actual at last month available'
            if 'ged_sb_run_average_6_ln1' or 'ged_ns_run_average_6_ln1' or 'ged_os_run_average_6_ln1' in variable: human_name = 'running average at last month available'
            title_run = f'Change of prediction from {human_name}, date {str(vid2date(month_id_temp))}, {var_name_run}'
            textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{cm_predictions_change_maps_folder}ChangeMaps_{dev_id}_cm_{var_run_savefile}_{region_name}_{cmap_run}_step{step}.png'
            
            if region_name in ('globe', 'ame', 'africa'):
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_layer(
                            gdf=data_run,
                        map_dictionary=dictionary_run,
                            cmap='binary',
                            edgecolor="black",
                            linewidth=0.5,
                            transparency = 1.0,
                            column=var_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = 1,
                            masking_location = region_name,
                            column=var_run,
                            edgecolor="black",
                            linewidth=1
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
            else:
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = country_mapping_transparency_global,
                            background = country_background_global,
                            masking_location = str(cid2name(region_name)),
                            column=var_run,
                            edgecolor="black",
                            linewidth=1, 
                            views_experimental_labels = views_experimental_font_global
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
print(f'{user}, cm prediction change maps from actuals completed')

### Continuos change maps, version 3 and 4 (old prediction, matching same step or same month)

In [ ]:
data = predictions_df.copy()

for variable_pred, variable_old in cm_change_variable_comparison.items():
    for months_back in months_back_comparison_predictions:
        column_prediction = variable_pred
        pred_old = 'cm_' + model_wanted + '_f' + str(EndOfHistory-months_back)
        temp_old = pd.DataFrame.forecasts.read_store(pred_old, run = dev_id_comparison)
        temp_old2 = temp_old.reset_index()
        temp_old2['step'] = temp_old2['month_id'] - (EndOfHistory-months_back)
        temp_old2.rename(columns =cm_change_variable_cont_conversion , inplace = True)
        
        temp_old_name = f'{variable_old}_mb_{months_back}'
        temp_old2[temp_old_name] = temp_old2[variable_old]
        column_comparison = temp_old_name

        for method_merge in {'step', 'month_id'}:
            data_merged = pd.merge(data.reset_index(), temp_old2.reset_index(), left_on = (method_merge, 'country_id'), right_on =(method_merge, 'country_id'), how = 'left', suffixes = ('','_y')).set_index(['step', 'country_id'])
            
            actual_prediction = np.exp(data_merged[column_prediction])-1
            actual_comparison = np.exp(data_merged[column_comparison])-1
            temp_change_name = f'change_from_{column_comparison}'
            data_merged[temp_change_name] = np.sign(actual_prediction - actual_comparison)*np.log(np.abs(actual_prediction - actual_comparison)+1)
            variable = temp_change_name
            
            gdf = gdf_ci_master.copy()
            data_merged = data_merged.join(gdf.set_index('country_id'))
            gdf = gpd.GeoDataFrame(data_merged, geometry = "geom")
            
            cm_predictions_change_maps_folder = master_folder + '/Continuous/Ensemble/ChangeMaps/cm/'
            
            steps_loop = steps_wanted_change_maps
            dictionary_run = change_dictionary_cm
            cmap_run = change_colormap_global
            
            
            for step in steps_loop:
                for region_name in geo_coverage_loop:
                    data_run = gdf.loc[step]
                    var_run = variable
                    var_name_run = find_the_violence_type(variable)
                    var_run_savefile = variable
                    bbox_run = bbox_from_cid(region_name)
                    month_id_temp = step+EndOfHistory

                    textbox_font_size = function_for_textbox(region_name)
                    title_run = f'Change of prediction from {pred_old} model {months_back} months back at same {method_merge} on {str(vid2date(month_id_temp))}, {var_name_run}'
                    textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_matched_by_{method_merge}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
                    savefile = f'{cm_predictions_change_maps_folder}ChangeMaps_{dev_id}_cm_vs_{pred_old}_variable_{var_run_savefile}_matched_by_{method_merge}_{region_name}_{cmap_run}_step{step}.png'
                    
                    if region_name in ('globe', 'ame', 'africa'):
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_layer(
                            gdf=data_run,
                        map_dictionary=dictionary_run,
                            cmap='binary',
                            edgecolor="black",
                            linewidth=0.5,
                            transparency = 1.0,
                            column=var_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = 1,
                            masking_location = region_name,
                            column=var_run,
                            edgecolor="black",
                            linewidth=1
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
                    else:
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = country_mapping_transparency_global,
                            background = country_background_global,
                            masking_location = str(cid2name(region_name)),
                            column=var_run,
                            edgecolor="black",
                            linewidth=1, 
                            views_experimental_labels = views_experimental_font_global
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
print(f'{user}, cm prediction maps from previous prediction completed')

### Dichotomous change maps

Unlike the previous example, there is no actuals to compare to, so we need to do only version 3 and 4 (old prediction, matching same step or same month)

Note that the 

In [ ]:
data = predictions_dich_df.copy()

for variable_pred, variable_old in cm_change_variable_comparison_dich.items():
    for months_back in months_back_comparison_predictions:
        column_prediction = variable_pred
        pred_old = 'cm_' + model_wanted + '_dich_f' + str(EndOfHistory-months_back)
        temp_old = pd.DataFrame.forecasts.read_store(pred_old, run = dev_id_comparison)
        temp_old2 = temp_old.reset_index()
        temp_old2['step'] = temp_old2['month_id'] - (EndOfHistory-months_back)
        temp_old2.rename(columns =cm_change_variable_dich_conversion , inplace = True)
        
        temp_old_name = f'{variable_old}_mb_{months_back}'
        temp_old2[temp_old_name] = temp_old2[variable_old]
        column_comparison = temp_old_name

        for method_merge in {'step', 'month_id'}:
            data_merged = pd.merge(data.reset_index(), temp_old2.reset_index(), left_on = (method_merge, 'country_id'), right_on = (method_merge, 'country_id'), how = 'left', suffixes = ('','_y')).set_index(['step', 'country_id'])
            
            actual_prediction =data_merged[column_prediction]
            actual_comparison = data_merged[column_comparison]
            temp_change_name = f'change_from_{column_comparison}'
            data_merged[temp_change_name] = np.sign(actual_prediction - actual_comparison)*(np.abs(actual_prediction - actual_comparison))
            variable = temp_change_name
            
            gdf = gdf_ci_master.copy()
            data_merged = data_merged.join(gdf.set_index('country_id'))
            gdf = gpd.GeoDataFrame(data_merged, geometry = "geom")
            
            cm_predictions_change_maps_folder = master_folder + '/Dichotomous/Ensemble/ChangeMaps/cm/'
            
            steps_loop = steps_wanted_change_maps
            dictionary_run = dichotomous_dictionary_change_cm
            cmap_run = change_colormap_global
            
            
            for step in steps_loop:
                for region_name in geo_coverage_loop:
                    data_run = gdf.loc[step]
                    var_run = variable
                    var_name_run = find_the_violence_type(variable)
                    var_run_savefile = variable
                    bbox_run = bbox_from_cid(region_name)
                    month_id_temp = step+EndOfHistory

                    textbox_font_size = function_for_textbox(region_name)
                    title_run = f'Change of prediction from {pred_old} model {months_back} months back at same {method_merge} on {str(vid2date(month_id_temp))}, {var_name_run}'
                    textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_matched_by_{method_merge}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
                    savefile = f'{cm_predictions_change_maps_folder}ChangeMaps_{dev_id}_cm_vs_{pred_old}_variable_{var_run_savefile}_matched_by_{method_merge}_{region_name}_{cmap_run}_step{step}.png'
                    
                    if region_name in ('globe', 'ame', 'africa'):
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_layer(
                            gdf=data_run,
                        map_dictionary=dictionary_run,
                            cmap='binary',
                            edgecolor="black",
                            linewidth=0.5,
                            transparency = 1.0,
                            column=var_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = 1,
                            masking_location = region_name,
                            column=var_run,
                            edgecolor="black",
                            linewidth=1
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
                    else:
                        map_run=Mapper2(
                            width=width_global,
                            height=height_global,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_mask(
                            gdf = data_run,
                            map_dictionary = dictionary_run,
                            cmap = cmap_run,
                            transparency = country_mapping_transparency_global,
                            background = country_background_global,
                            masking_location = str(cid2name(region_name)),
                            column=var_run,
                            edgecolor="black",
                            linewidth=1, 
                            views_experimental_labels = views_experimental_font_global
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)

                        map_run.save(savefile)
print(f'{user}, cm prediction maps for dich predictions from previous prediction completed')

# Prediction, pgm level

In [ ]:
#read in future predictions at pgm level from internal storage

predstore_future = 'pgm_' + model_wanted_pgm + '_f' + str(EndOfHistory)
predstore_future_dich = 'pgm_' +  model_wanted_pgm + '_dich_f' + str(EndOfHistory)

predictions_df = pd.DataFrame.forecasts.read_store(predstore_future, run=dev_id)
predictions_df = predictions_df.reset_index()
predictions_df['step'] = predictions_df['month_id'] - EndOfHistory
predictions_df = predictions_df.set_index(['step', 'priogrid_id'], drop = True)
predictions_df.rename(columns = prediction_cont_name_default, inplace = True)
if do_we_need_to_double_log_cont_variable == 'yes':
    for i in prediction_cont_name_double_logged: predictions_df[i['new_name']]=np.log1p(predictions_df[i['old_name']])

predictions_dich_df = pd.DataFrame.forecasts.read_store(predstore_future_dich, run=dev_id)
predictions_dich_df = predictions_dich_df.reset_index()
predictions_dich_df['step'] = predictions_dich_df['month_id'] - EndOfHistory
predictions_dich_df = predictions_dich_df.set_index(['step', 'priogrid_id'], drop = True)
predictions_dich_df.rename(columns = prediction_dich_name_default, inplace = True)

print(f'{user}, prediction data at cm level successfully fetched')

### Continuous prediction

In [ ]:
#for predictions, we have chosen steps we wanted earlier

#prep data
#this process took a while and we technically just need the end of history month
data= predictions_df.copy().reset_index().set_index(['priogrid_id'])
gdf = gdf_pid_master.copy().query(f"month_id == {EndOfHistory}").reset_index().drop('month_id', axis = 1).rename(columns = {'priogrid_gid':'priogrid_id'}).set_index(['priogrid_id'])

data_pid = pd.merge(left = data, right = gdf, left_index = True, right_index = True, how= 'left').reset_index().set_index(['step', 'priogrid_id'])

gdf = gpd.GeoDataFrame(data_pid, geometry="geom")
gdf_c = gdf_ci_master.copy()

#savefolder
predictions_pgm_folder = master_folder+'/Continuous/Ensemble/ForecastMaps/pgm/'

#looping informationa
variable_loop = variables_wanted_predictions
dictionary_run = fatality_dictionary_pgm
cmap_run = fatality_colormap
steps_loop = steps_wanted_prediction_fat

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = give_me_violence_string_label_only(variable)
            bbox_run = bbox_from_cid_region(region_name)
            month_id = step+EndOfHistory
                        
            #making the choice of textbox size
            textbox_font_size = function_for_textbox(region_name)
            
             #masking_choices
            if str(region_name) in('globe'):
                masking_run = 'globe'
                background2 = None
            elif str(region_name) in ('ame'):
                masking_run = 'ame'
                background2 = None
            elif str(region_name) in ('africa'):
                masking_run = 'africa'
                background2 = None
            else:
                masking_run = cid2name(region_name)
                background2 = pgm_background_country_level
                        
            title_run = f'{prediction_title_name} in {str(vid2date(month_id))}, {var_name_run}'
            textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{predictions_pgm_folder}{dev_id}_cm_{var_run_savefile}_{region_name}_{cmap_run}_step{step}.png'
            
            
            masked=Mapper2(
                width=width_global,
                height=height_global,
                frame_on=True,
                title=title_run,
                bbox=bbox_run
            ).add_mask(
                gdf = data_run,
                map_dictionary = dictionary_run,
                cmap = cmap_run,
                transparency = 1,
                masking_location = masking_run,
                column=var_run,
                background = background2,
                edgecolor="black",
                linewidth=0.5
            ).add_views_textbox(
                text=textbox,
                textsize=textbox_font_size)
            
            ax = masked.ax
            
            masked_with_borders = gdf_c.plot(ax=ax, edgecolor = 'black', linewidth = 2.0, facecolor = 'None')
            plt.savefig(savefile, dpi = pgm_dpi_global)
            
            plt.close()
print(f'{user}, pgm prediction maps completed')

### Dichotomous prediction

In [ ]:
#for predictions, we have chosen steps we wanted earlier

#prep data
#this process took a while and we technically just need the end of history month
data= predictions_dich_df.copy().reset_index().set_index(['priogrid_id'])
gdf = gdf_pid_master.copy().query(f"month_id == {EndOfHistory}").reset_index().drop('month_id', axis = 1).rename(columns = {'priogrid_gid':'priogrid_id'}).set_index(['priogrid_id'])

data_pid = pd.merge(left = data, right = gdf, left_index = True, right_index = True, how= 'left').reset_index().set_index(['step', 'priogrid_id'])

gdf = gpd.GeoDataFrame(data_pid, geometry="geom")
gdf_c = gdf_ci_master.copy()

#savefolder
predictions_pgm_folder = master_folder+'/Dichotomous/Ensemble/ForecastMaps/pgm/'

#looping informationa
variable_loop = variables_wanted_predictions_dich
dictionary_run = dictionary_dichotomous_pgm
cmap_run = fatality_colormap
steps_loop = steps_wanted_prediction_fat

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = give_me_violence_string_label_only(variable)
            bbox_run = bbox_from_cid_region(region_name)
            month_id = step+EndOfHistory
                        
            #making the choice of textbox size
            textbox_font_size = function_for_textbox(region_name)
            
             #masking_choices
            if str(region_name) in('globe'):
                masking_run = 'globe'
                background2 = None
            elif str(region_name) in ('ame'):
                masking_run = 'ame'
                background2 = None
            elif str(region_name) in ('africa'):
                masking_run = 'africa'
                background2 = None
            else:
                masking_run = cid2name(region_name)
                background2 = pgm_background_country_level
                        
            title_run = f'{prediction_title_name} in {str(vid2date(month_id))}, {var_name_run}'
            textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{predictions_pgm_folder}{dev_id}_cm_{var_run_savefile}_{region_name}_{cmap_run}_step{step}.png'
            
            
            masked=Mapper2(
                width=width_global,
                height=height_global,
                frame_on=True,
                title=title_run,
                bbox=bbox_run
            ).add_mask(
                gdf = data_run,
                map_dictionary = dictionary_run,
                cmap = cmap_run,
                transparency = 1,
                masking_location = masking_run,
                column=var_run,
                background = background2,
                edgecolor="black",
                linewidth=0.5
            ).add_views_textbox(
                text=textbox,
                textsize=textbox_font_size)
            
            ax = masked.ax
            
            masked_with_borders = gdf_c.plot(ax=ax, edgecolor = 'black', linewidth = 2.0, facecolor = 'None')
            plt.savefig(savefile, dpi = pgm_dpi_global)
            
            plt.close()
print(f'{user}, pgm prediction dich maps completed')

### Continuous change version 1-2

In [ ]:
#continuous_changes_vs_actuals_or_running_average
data = predictions_df.copy()
historic = Queryset("Monthly_update_features_pgm", "priogrid_month").fetch()[['ged_sb_ln1', 'ged_ns_ln1', 'ged_os_ln1']]
end_of_history = historic.query(f'month_id == {EndOfHistory}')
data2 = pd.merge(data.reset_index(), end_of_history, left_on = 'priogrid_id', right_on = 'priogrid_gid', how = 'left').set_index(['step', 'priogrid_id'])
rolling_average = Queryset("Monthly_update_running_average_pgm", "country_month").fetch()
rolling_average_end_of_history = rolling_average.query(f'month_id == {EndOfHistory}')
data3 = pd.merge(data2.reset_index(), rolling_average_end_of_history, left_on = 'priogrid_id', right_on = 'priogrid_gid', how = 'left').set_index(['step', 'priogrid_id'])

for variable, variable_old in cm_change_variable_comparison.items():
    conf = give_me_violence_string_label_only(variable)
    if conf == '_sb': variables_to_compare = {'ged_sb_ln1', 'ged_sb_run_average_6_ln1'}
    elif conf == '_ns': variables_to_compare = {'ged_ns_ln1', 'ged_ns_run_average_6_ln1'}
    elif conf == '_os': variables_compare = {'ged_os_ln1', 'ged_os_run_average_6_ln1'}
    else: print('variable name does not specify violence type, make sure it does')
    column_prediction = variable
    variable_loop = []
    for column_comparison in variables_to_compare:
        actual_prediction = np.exp(data3[column_prediction])-1
        actual_comparison = np.exp(data3[column_comparison])-1
        temp_change_name = f'change_from_{column_comparison}'
        data3[temp_change_name] = np.sign(actual_prediction - actual_comparison)*np.log(np.abs(actual_prediction - actual_comparison)+1)
        variable_loop.append(temp_change_name)

        
#geometries
gdf = gdf_pid_master.copy().query(f"month_id == {EndOfHistory}").reset_index().drop('month_id', axis = 1).rename(columns = {'priogrid_gid':'priogrid_id'}).set_index(['priogrid_id'])
data4 = pd.merge(data3.reset_index().set_index(['priogrid_id']), gdf.reset_index().set_index(['priogrid_id']), left_index = True, right_index = True, how = 'left').reset_index().set_index(['step', 'priogrid_id'])
gdf = gpd.GeoDataFrame(data4, geometry = "geom")
gdf_c = gdf_ci_master.copy()

cm_predictions_change_maps_folder = master_folder + '/Continuous/Ensemble/ChangeMaps/pgm/'

steps_loop = steps_wanted_change_maps
dictionary_run = change_dictionary_pgm
cmap_run = change_colormap_global

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[step]
            var_run = variable
            var_name_run = find_the_violence_type(variable)
            var_run_savefile = variable
            bbox_run = bbox_from_cid(region_name)
            month_id_temp = step+EndOfHistory

            textbox_font_size = function_for_textbox(region_name)
            
             #masking_choices
            if str(region_name) in('globe'):
                masking_run = 'globe'
                background2 = None
            elif str(region_name) in ('ame'):
                masking_run = 'ame'
                background2 = None
            elif str(region_name) in ('africa'):
                masking_run = 'africa'
                background2 = None
            else:
                masking_run = cid2name(region_name)
                background2 = pgm_background_country_level

            if 'ged_sb_ln1' or 'ged_ns_ln1' or 'ged_os_ln1' in variable: human_name = 'actual at last month available'
            if 'ged_sb_run_average_6_ln1' or 'ged_ns_run_average_6_ln1' or 'ged_os_run_average_6_ln1' in variable: human_name = 'running average at last month available'
            title_run = f'Change of prediction from {human_name}, date {str(vid2date(month_id_temp))}, {var_name_run}'
            textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
            savefile = f'{cm_predictions_change_maps_folder}ChangeMaps_{dev_id}_cm_{var_run_savefile}_{region_name}_{cmap_run}_step{step}.png'
            
            masked=Mapper2(
                width=width_global,
                height=height_global,
                frame_on=True,
                title=title_run,
                bbox=bbox_run
            ).add_mask(
                gdf = data_run,
                map_dictionary = dictionary_run,
                cmap = cmap_run,
                transparency = 1,
                masking_location = masking_run,
                column=var_run,
                background = background2,
                edgecolor="black",
                linewidth=0.5
            ).add_views_textbox(
                text=textbox,
                textsize=textbox_font_size)
            
            ax = masked.ax
            
            masked_with_borders = gdf_c.plot(ax=ax, edgecolor = 'black', linewidth = 2.0, facecolor = 'None')
            plt.savefig(savefile, dpi = pgm_dpi_global)
            
            plt.close()
            
print(f'{user}, pgm prediction change maps from actuals completed')

### Continous change version 3-4

In [ ]:
data = predictions_df.copy()

for variable_pred, variable_old in cm_change_variable_comparison.items():
    for months_back in months_back_comparison_predictions:
        column_prediction = variable_pred
        pred_old = 'pgm_' + model_wanted_pgm + '_f' + str(EndOfHistory-months_back)
        temp_old = pd.DataFrame.forecasts.read_store(pred_old, run = dev_id_comparison)
        temp_old2 = temp_old.reset_index()
        temp_old2['step'] = temp_old2['month_id'] - (EndOfHistory-months_back)
        temp_old2.rename(columns =cm_change_variable_cont_conversion , inplace = True)
        
        temp_old_name = f'{variable_old}_mb_{months_back}'
        temp_old2[temp_old_name] = temp_old2[variable_old]
        column_comparison = temp_old_name
        
        for method_merge in {'step', 'month_id'}:
            data_merged = pd.merge(data.reset_index(), temp_old2.reset_index(), left_on = (method_merge, 'priogrid_id'), right_on = (method_merge, 'priogrid_id'), how = 'left', suffixes = ('','_y')).set_index(['step', 'priogrid_id'])
            
            actual_prediction = np.exp(data_merged[column_prediction])-1
            actual_comparison = np.exp(data_merged[column_comparison])-1
            temp_change_name = f'change_from_{column_comparison}'
            data_merged[temp_change_name] = np.sign(actual_prediction - actual_comparison)*np.log(np.abs(actual_prediction - actual_comparison)+1)
            variable = temp_change_name
            
            gdf = gdf_pid_master.copy().query(f"month_id == {EndOfHistory}").reset_index().drop('month_id', axis = 1).rename(columns = {'priogrid_gid':'priogrid_id'}).set_index(['priogrid_id'])
            data_merged2 = pd.merge(data_merged.reset_index().set_index(['priogrid_id']), gdf.reset_index().set_index(['priogrid_id']), left_index = True, right_index = True, how = 'left').reset_index().set_index(['step', 'priogrid_id'])
            gdf = gpd.GeoDataFrame(data_merged2, geometry = "geom")
            gdf_c = gdf_ci_master.copy()
            
            cm_predictions_change_maps_folder = master_folder + '/Continuous/Ensemble/ChangeMaps/pgm/'
            
            steps_loop = steps_wanted_change_maps
            dictionary_run = change_dictionary_pgm
            cmap_run = change_colormap_global
            
            
            for step in steps_loop:
                for region_name in geo_coverage_loop:
                    data_run = gdf.loc[step]
                    var_run = variable
                    var_name_run = find_the_violence_type(variable)
                    var_run_savefile = variable
                    bbox_run = bbox_from_cid(region_name)
                    month_id_temp = step+EndOfHistory

                    textbox_font_size = function_for_textbox(region_name)
                    
                    #masking_choices
                    if str(region_name) in('globe'):
                        masking_run = 'globe'
                        background2 = None
                    elif str(region_name) in ('ame'):
                        masking_run = 'ame'
                        background2 = None
                    elif str(region_name) in ('africa'):
                        masking_run = 'africa'
                        background2 = None
                    else:
                        masking_run = cid2name(region_name)
                        background2 = pgm_background_country_level
                    
                    title_run = f'Change of prediction from {pred_old} model {months_back} months back at same {method_merge} on {str(vid2date(month_id_temp))}, {var_name_run}'
                    textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_matched_by_{method_merge}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
                    savefile = f'{cm_predictions_change_maps_folder}ChangeMaps_{dev_id}_cm_vs_{pred_old}_variable_{var_run_savefile}_matched_by_{method_merge}_{region_name}_{cmap_run}_step{step}.png'
                    
                    masked=Mapper2(
                        width=width_global,
                        height=height_global,
                        frame_on=True,
                        title=title_run,
                        bbox=bbox_run
                    ).add_mask(
                        gdf = data_run,
                        map_dictionary = dictionary_run,
                        cmap = cmap_run,
                        transparency = 1,
                        masking_location = masking_run,
                        column=var_run,
                        background = background2,
                        edgecolor="black",
                        linewidth=0.5
                    ).add_views_textbox(
                        text=textbox,
                        textsize=textbox_font_size)

                    ax = masked.ax

                    masked_with_borders = gdf_c.plot(ax=ax, edgecolor = 'black', linewidth = 2.0, facecolor = 'None')
                    plt.savefig(savefile, dpi = pgm_dpi_global)

                    plt.close()
                    
                    
print(f'{user}, pgm prediction maps from previous prediction completed')

### Dichotomous change version 3-4

note again, only can compare to other predictions

In [ ]:
data = predictions_dich_df.copy()

for variable_pred, variable_old in cm_change_variable_comparison_dich.items():
    for months_back in months_back_comparison_predictions:
        column_prediction = variable_pred
        pred_old = 'pgm_' + model_wanted_pgm + '_dich_f' + str(EndOfHistory-months_back)
        temp_old = pd.DataFrame.forecasts.read_store(pred_old, run = dev_id_comparison)
        temp_old2 = temp_old.reset_index()
        temp_old2['step'] = temp_old2['month_id'] - (EndOfHistory-months_back)
        temp_old2.rename(columns =cm_change_variable_dich_conversion , inplace = True)
        
        temp_old_name = f'{variable_old}_mb_{months_back}'
        temp_old2[temp_old_name] = temp_old2[variable_old]
        column_comparison = temp_old_name
        
        for method_merge in {'step', 'month_id'}:
            data_merged = pd.merge(data.reset_index(), temp_old2.reset_index(), left_on = (method_merge, 'priogrid_id'), right_on = (method_merge, 'priogrid_id'), how = 'left', suffixes = ('','_y')).set_index(['step', 'priogrid_id'])
            
            actual_prediction = data_merged[column_prediction]
            actual_comparison = data_merged[column_comparison]
            temp_change_name = f'change_from_{column_comparison}'
            data_merged[temp_change_name] = np.sign(actual_prediction - actual_comparison)*(np.abs(actual_prediction - actual_comparison))
            variable = temp_change_name
            
            gdf = gdf_pid_master.copy().query(f"month_id == {EndOfHistory}").reset_index().drop('month_id', axis = 1).rename(columns = {'priogrid_gid':'priogrid_id'}).set_index(['priogrid_id'])
            data_merged2 = pd.merge(data_merged.reset_index().set_index(['priogrid_id']), gdf.reset_index().set_index(['priogrid_id']), left_index = True, right_index = True, how = 'left').reset_index().set_index(['step', 'priogrid_id'])
            gdf = gpd.GeoDataFrame(data_merged2, geometry = "geom")
            gdf_c = gdf_ci_master.copy()
            
            cm_predictions_change_maps_folder = master_folder + '/Dichotomous/Ensemble/ChangeMaps/pgm/'
            
            steps_loop = steps_wanted_change_maps
            dictionary_run = dichotomous_dictionary_change_pgm
            cmap_run = change_colormap_global
            
            
            for step in steps_loop:
                for region_name in geo_coverage_loop:
                    data_run = gdf.loc[step]
                    var_run = variable
                    var_name_run = find_the_violence_type(variable)
                    var_run_savefile = variable
                    bbox_run = bbox_from_cid(region_name)
                    month_id_temp = step+EndOfHistory

                    textbox_font_size = function_for_textbox(region_name)
                    
                    #masking_choices
                    if str(region_name) in('globe'):
                        masking_run = 'globe'
                        background2 = None
                    elif str(region_name) in ('ame'):
                        masking_run = 'ame'
                        background2 = None
                    elif str(region_name) in ('africa'):
                        masking_run = 'africa'
                        background2 = None
                    else:
                        masking_run = cid2name(region_name)
                        background2 = pgm_background_country_level
                    
                    title_run = f'Change of prediction from {pred_old} model {months_back} months back at same {method_merge} on {str(vid2date(month_id_temp))}, {var_name_run}'
                    textbox = f'Name: {dev_id}_{model_wanted}_{model_attempt},\n{var_run}_matched_by_{method_merge}_at_step{str(step)},\nlast input: {str(vid2date(EndOfHistory))}'
                    savefile = f'{cm_predictions_change_maps_folder}ChangeMaps_{dev_id}_cm_vs_{pred_old}_variable_{var_run_savefile}_matched_by_{method_merge}_{region_name}_{cmap_run}_step{step}.png'
                    
                    masked=Mapper2(
                        width=width_global,
                        height=height_global,
                        frame_on=True,
                        title=title_run,
                        bbox=bbox_run
                    ).add_mask(
                        gdf = data_run,
                        map_dictionary = dictionary_run,
                        cmap = cmap_run,
                        transparency = 1,
                        masking_location = masking_run,
                        column=var_run,
                        background = background2,
                        edgecolor="black",
                        linewidth=0.5
                    ).add_views_textbox(
                        text=textbox,
                        textsize=textbox_font_size)

                    ax = masked.ax

                    masked_with_borders = gdf_c.plot(ax=ax, edgecolor = 'black', linewidth = 2.0, facecolor = 'None')
                    plt.savefig(savefile, dpi = pgm_dpi_global)

                    plt.close()
                    
                    
print(f'{user}, pgm prediction dichotomous maps from previous prediction completed')